In [ ]:
# Before anything, run this cell first
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.2 MB

In [ ]:
!pip install keras_resnet

  Preparing metadata (setup.py) ... done
  Created wheel for keras_resnet: filename=keras_resnet-0.2.0-py2.py3-none-any.whl size=20457 sha256=9c1c6f6f2fd5a87174c4d0e62ba255e1df85d750c416d2b1033e682d1bc286d7
  Stored in directory: /root/.cache/pip/wheels/16/af/88/a668b279c5eadbe55dcaf6207f09059135166cefb09088bacc
Successfully built keras_resnet


In [ ]:
import os
os.chdir("/content/drive/My Drive/ME781/Project/keras-retinanet")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from termcolor import colored
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
import keras_resnet
import cv2

In [ ]:
def defect_detection(image_file):
  model_file_path = "/content/drive/MyDrive/ME781/Project/defect_detection/defect_detection_vgg16.hdf5"
  model = load_model(model_file_path)

  # setting image dimensions
  image_width = 224
  image_height = 224
  labels = {0: "Defective", 1: "Not Defective"}

  # load and pre-process the image
  img = image.load_img(image_file, target_size=(image_width, image_height))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)

  # run the image through the defect detection model to obtain the predictions

  out = model.predict(x)
  pred = np.argmax(out[0])
  return labels[pred]

In [ ]:
def defect_classification(image_file):
  model_file_path = "/content/drive/MyDrive/ME781/Project/defect_classification/defect_classification_vgg16.hdf5"
  model = load_model(model_file_path)

  # setting image dimensions
  image_width = 224
  image_height = 224
  labels = {0: "Crazing", 1: "Inclusion", 2: "Patches", 3: "Pitted", 4: "Rolled", 5: "Scratches"}

  # load and pre-process the image
  img = image.load_img(image_file, target_size=(image_width, image_height))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)

  # run the image through the defect detection model to obtain the predictions

  out = model.predict(x)
  pred = np.argmax(out[0])
  return labels[pred]

In [ ]:
def object_detection(image_file, labels_to_names = {0: "pump_impeller"}):

  model_file_path = "/content/drive/MyDrive/ME781/Project/object_detection/pump_impeller_detection_retinanet.h5"
  model = models.load_model(model_file_path, backbone_name = "resnet50")
  threshold = 0.7

  # read and pre-process the image
  image = read_image_bgr(image_file)
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
  image = preprocess_image(image)
  image, scale = resize_image(image)
  x = np.expand_dims(image, axis = 0)

  # obtain the bounding boxes for the objects detected in the image and correct their scale
  boxes, scores, labels = model.predict_on_batch(x)
  boxes /= scale

  # store the bounding box details for valid detection (i.e. confidence > threshold)
  bounding_boxes = {}
  count = 0
  for box, score, label in zip(boxes[0], scores[0], labels[0]):

    # scores are sorted in descending order hence we can 'break' out of the loop when we reach below threshold for the first time
    if score < threshold:
      break

    color = label_color(label)

    b = box.astype(int)
    draw_box(draw, b, color=color)

    caption = "{} {:.3f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)

  return draw

In [ ]:
import gradio as gr

defect_detection_interface = gr.Interface(fn = defect_detection, inputs = gr.Image(type="filepath"), outputs="text", title="Identify and elimate product defects using computer vision and AI", description="Check out the glimse of our defect detection model which uses computer vision and CNNs to identify the relevant parts from an image and classify them to be either defective or good quality.")
defect_classification_interface = gr.Interface(fn = defect_classification, inputs = gr.Image(type="filepath"), outputs="text", title="Localize and classify the type of defects in manufactured parts", description="Try out our defect classifier model which can predict the types of defect that a product may have based on the image using deep CNNs trained using transfer learning. This feature allows you to classify the different types of defects on metal surfaces into 6 different categories.")
object_detection_interface = gr.Interface(fn = object_detection, inputs = gr.Image(type="filepath"), outputs="image", title="Identify the presence of an object using computer vision and AI", description="Check out our object detection model which leverages CNNs to identify the presence of an object and classifies it accordingly")

In [ ]:
final_interface = gr.TabbedInterface(interface_list=[object_detection_interface, defect_detection_interface, defect_classification_interface],
                         tab_names = ["Object Detection Interface", "Defect Detection Interface", "Defect Classification Interface"],
                                     title="MechDefect Solutions")

if __name__ == '__main__':
    final_interface.launch(share=True, inline = False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://daac75c92c7df59721.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
